# 台灣外來入侵種資料集 工具箱

V0.1

## Init All 

In [3]:
from tiasdswak import *
from codes.lib import *

tiasdswak = TiasdSwak()

df_standard = tiasdswak.df_standard # 政府資料
df_append = tiasdswak.df_append # 民間資料庫
df_extend = tiasdswak.df_extend # 補充資料
df_mix = tiasdswak.df_mix # 政府+民間
df_merge = tiasdswak.df_merge # 政府+民間+補充
df_cur = tiasdswak.df_cur # 目前查詢結果
df_dig = tiasdswak.df_dig # 分析


TiasdSWAK-V0.1


In [ ]:
#df_dig['new']=1
#df_dig
#print("\n".join(list(df_merge.columns)))

#str1="','".join(tiasdswak.group['十大外來入侵物種'])
#str2="'%s'" %(str1)
#str2

#sql="select * from df"
#df1=sqldf(df_merge,sql)

#df_cur['name_code'].tolist()
#grp_info={"大花咸豐草":"9164c378-ef51-4423-a166-1a979c73a97d"}
#grp_info={"中國梨木蝨":"f7d026a6-719c-4c1b-ac04-f9e18b0a8abb"}
if 0:
    #df.to_csv("output/tbn_save.csv",index=False)
    df_merge.to_csv("output/台灣外來入侵種資料集 - 合併補充資料集.csv",index=False)
    
df_merge

## MIX 

In [ ]:
df_mix = df_standard
for row in df_append.iterrows():
    name_code = str(row[1]['name_code'])
    print(name_code)
    name_code_values=df_mix[df_mix['name_code']==name_code]['name_code'].values.tolist()
    if len(name_code_values) > 0: #delete match row
        print("%s exist!" %(name_code))
        idx=df_mix.index[df_mix['name_code']==name_code].tolist()
        df_mix.drop(idx, axis=0, inplace=True) 
df_mix=df_mix[df_mix['is_invasive']==1].append(df_append[df_append['is_invasive']==1])


## MERGE 

In [ ]:
df_extend = df_extend.astype({'name_code':'str'})
df_merge = df_mix.join(df_extend.set_index('name_code'),on='name_code',how='outer',lsuffix='L',rsuffix='R')
#df_merge.to_csv("/Users/wuulong/prg/tiasdswak/include/台灣外來入侵種資料庫補充資料集 - 合併補充資料集.csv",index=False)

In [ ]:
results=[]
for row in df_merge.iterrows():
    common_name_c = str(row[1]['common_name_c'])
    name_code = str(row[1]['name_code'])
    print("common_name_c=%s,name_code=%s" %(common_name_c,name_code))
    [taxonUUID,total]=getcnt_by_namecode(name_code)
    result=[common_name_c,name_code,taxonUUID,total]
    print(result)
    results.append(result)

In [ ]:
for result in results:
    print("%s,%s,%s,%s" %(result[0],result[1],result[2],result[3]))

# 多選展開

In [ ]:
# 多選展開
# ex: 都市區, 農業區
col_sel=['棲地類型Habitat types','類別Life form types']

def multisel_expand(df,col_sel):
    results = []
    for col_name in col_sel:
        df_col=df[col_name]
        cells = df_col[df_col.notna()].tolist()
        for cell in cells:
            values = cell.split(",")
            values = list(map(str.strip,values))
            for v in values:
                result = [col_name,v]
                results.append(result)
    return results

results = multisel_expand(df_extend,col_sel)

for r in results:
    print("%s,%s" %(r[0],r[1]))

# 多選項目編碼

In [ ]:
items_str="""具入侵性
不具入侵性
無特定說明
不確定


"""



In [ ]:
items = items_str.split("\n")
i=1
for item in items:
    if len(item)>0:
        cols=item.split("-")
        if len(cols)==2:
            print("%i-%s" %(i,cols[1].strip()))
        else:
            print("%i-%s" %(i,item.strip()))
        i+=1

In [ ]:
txt="""BaseElevation(基準高)、County(縣市名稱)、CountyIdentifier(縣市代碼)、DisuseDate(廢站日期)、EstablishDate(設站日期)、FinishDate(完鑿日期)、FinishDepth(完鑿深度)、GroundHeight(距地面高)、GroundwaterLayerCode(地下水分層代碼)、GroundwaterZone(地下水分區名稱)、GroundwaterZoneIdentifier(地下水分區代碼)、Identifier(井號)、LayerAttribute(含水層屬性，C為受壓 L為漏壓 U為自由)、LocationAddress(站址)、LocationByTWD67(TM2度分帶67坐標)、LocationByTWD97(TM2度分帶97座標，TM2度分帶97坐標)、ModificationDate(異動日期)、ModificationReason(異動別，1為設站 2為遷站 3為撤站 4為站況異動 5為其他異動 6為復站)、OrganizationIdentifier(管理單位代碼)、PipeMaterial(井管材質，1為不鏽鋼管 2為PVC4 3為鍍鋅鋼管 4為鋼管 5為鐵管 6為塑膠管 7為黑鐵管)、Remarks(備註)、StartDateOfWaterLevelObservation(水位開始觀測日期)、Town(鄉鎮名稱)、TownIdentifier(鄉鎮代碼)、WaterLevel(水位高)、WaterResourcesDistrictName(水資源分區名稱)、WellDepth(井深)、WellElevation(目前井頂高)、WellName(井名稱)、WellNumber(孔數)、WellRadius(井徑)、WellType(井別)"""
cols=txt.split('、')
for col in cols:
    print(col.strip().replace(" ",""))

# Generate Report

In [ ]:

def rpt_gen(df_rows):
    lines=[]
    for row in df_rows.iterrows():
        name_code = str(row[1]['name_code'])
        common_name_c = str(row[1]['common_name_c'])
        print("# %s (%s)" % (name_code,common_name_c))
        row1 = row[1]
        print("| 欄位名稱 | 欄位值 |")
        print("| ------------- |:-------------|")
        for i in range(len(row1.index)):
            if str(row1.values[i]).find("\n")>0:
                lines.append("## %s" %(row1.index[i]))
                lines.append("%s" %(str(row1.values[i]).replace("\n","\n\n")))
            #if row1.index[i] not in ('reference','note','dispersal_mechanisms','description.1','summary'):
            else:
                print("| %s | %s |" %(row1.index[i],row1.values[i]))
    for line in lines:
        print(line)

idx=df_merge.index[df_merge['name_code']=='419665']
df_rows = df_merge.loc[idx]
rpt_gen(df_rows)